In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
data= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movies.csv')
data.head()

,review,sentiment,sentiments
0,One of the other reviewers has mentioned that ...,positive,1
1,A wonderful little production. <br /><br />The...,positive,1
2,I thought this was a wonderful way to spend ti...,positive,1
3,Basically there's a family where a little boy ...,negative,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1


In [ ]:
import gzip
import shutil
import time

import pandas as pd
import requests
import torch
import torch.nn.functional as F
import torchtext

import transformers
from transformers import DistilBertForSequenceClassification
from transformers import DistilBertTokenizerFast

In [ ]:
NUM_EPOCHS=3
DEVICE=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(data.shape)

(14999, 3)


In [ ]:
train_texts=data.iloc[:8000] ['review'].values
train_labels=data.iloc[:8000] ['sentiments'].values
valid_texts=data.iloc[8000:10000] ['review'].values
valid_labels=data.iloc[8000:10000] ['sentiments'].values
test_texts=data.iloc[10000:] ['review'].values
test_labels=data.iloc[10000:] ['sentiments'].values

In [ ]:
tokenizer=DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
train_encodings=tokenizer(list(train_texts), truncation=True, padding=True)
valid_encodings=tokenizer(list(valid_texts), truncation=True, padding=True)
test_encodings=tokenizer(list(test_texts), truncation=True, padding=True)

In [ ]:
train_encodings[0]

Encoding(num_tokens=512, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [ ]:
class IMDbDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self. labels = labels
  def __getitem__(self, idx):
    item={key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(int(self.labels [idx]))
    return item
  def __len__ (self):
    return len(self.labels)

train_dataset=IMDbDataset (train_encodings, train_labels)
valid_dataset=IMDbDataset (valid_encodings, valid_labels)
test_dataset=IMDbDataset(test_encodings, test_labels)

In [ ]:
train_loader=torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader=torch.utils.data.DataLoader(valid_dataset, batch_size=16, shuffle=False)
test_loader=torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
model=DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model.to(DEVICE)
model.train()
optim=torch.optim.Adam(model.parameters(), lr=5e-5)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def compute_accuracy(model, data_loader, device):
  with torch.no_grad():
    correct_pred, num_examples = 0, 0
    for batch_idx, batch in enumerate(data_loader):
    ### Prepare data
      input_ids=batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to (device)
      labels= batch['labels'].to (device)
      outputs=model(input_ids, attention_mask=attention_mask, labels=labels)
      loss, logits = outputs ['loss'], outputs ['logits']
      _,predicted_labels = torch.max(logits, 1)
      num_examples += labels.size(0)
      correct_pred += (predicted_labels == labels).sum()
  return correct_pred.float()/num_examples * 100

In [ ]:
start_time = time.time()
for epoch in range(NUM_EPOCHS):
  model.train()
  for batch_idx, batch in enumerate (train_loader):
    ### Prepare data
    input_ids=batch['input_ids'].to(DEVICE)
    attention_mask=batch['attention_mask'].to (DEVICE)
    labels=batch['labels'].to (DEVICE)
    ### Forward
    outputs=model(input_ids, attention_mask=attention_mask, labels=labels)
    loss, logits=outputs ['loss'], outputs ['logits']
    ### Backward
    optim.zero_grad()
    loss.backward()
    optim.step()

    ### Logging
    if not batch_idx % 250:
      print (f'Epoch: {epoch+1:04d}/{NUM_EPOCHS:04d} | '
      f'Batch {batch_idx:04d}/{len(train_loader):04d} | '
      f'Loss: {loss:.4f}')
  model.eval()
  with torch.set_grad_enabled(False):
    print(f'training accuracy:'
    f'{compute_accuracy (model, train_loader, DEVICE):.2f}%'
    f'\nvalid accuracy:'
    f'{compute_accuracy (model, valid_loader, DEVICE):.2f}%')
  print(f'Time elapsed: {(time.time()-start_time)/60:.2f} min')
print(f'Total Training Time: {(time.time()-start_time)/60:.2f} min')
print(f'Test accuracy: {compute_accuracy(model, test_loader, DEVICE):.2f}%')

Epoch: 0001/0003 | Batch 0000/0500 | Loss: 0.6841
Epoch: 0001/0003 | Batch 0250/0500 | Loss: 0.4274
training accuracy:95.05%
valid accuracy:89.30%
Time elapsed: 31.98 min
Epoch: 0002/0003 | Batch 0000/0500 | Loss: 0.0353
Epoch: 0002/0003 | Batch 0250/0500 | Loss: 0.0109
training accuracy:95.78%
valid accuracy:88.30%
Time elapsed: 61.42 min
Epoch: 0003/0003 | Batch 0000/0500 | Loss: 0.2368
Epoch: 0003/0003 | Batch 0250/0500 | Loss: 0.0527
training accuracy:99.58%
valid accuracy:90.80%
Time elapsed: 90.89 min
Total Training Time: 90.89 min
Test accuracy: 90.66%


In [ ]:
# After training the model
model.save_pretrained('/content/drive/MyDrive/Colab Notebooks/saved_model')
tokenizer.save_pretrained('/content/drive/MyDrive/Colab Notebooks/saved_tokenizer')

# Optionally, save the optimizer state if you want to resume training later
torch.save(optim.state_dict(), '/content/drive/MyDrive/Colab Notebooks/saved_optimizer.pth')
